# Hubbard Model Part I - Dhilan Nag

In [152]:
from itertools import permutations as perm

#Model parameters
siteNum = int(input("# of sites: "))
upNum = int(input("# of spin up electrons: "))
downNum = int(input("# of spin down electrons: "))

# of sites: 2
# of spin up electrons: 1
# of spin down electrons: 1


In [153]:
#Create bit strings

#Spin up
upStrings = []
for i in range(upNum):
    upStrings.append("1")
for i in range(siteNum-upNum):
    upStrings.append("0")

upStrings = list(set(perm(upStrings,siteNum)))

#Spin down
downStrings = []
for i in range(downNum):
    downStrings.append("1")
for i in range(siteNum-downNum):
    downStrings.append("0")

downStrings = list(set(perm(downStrings,siteNum)))

#Set up string output for bit strings    
output = "|"
for up in upStrings:
    for down in downStrings:
        for b in up:
            output += b+","
        output = output[0:len(output)-1]+">|"
        for b in down:
            output += b+","
        output = output[0:len(output)-1]+">\n|"
output = output[0:len(output)-1]
    

print(output)

|1,0>|1,0>
|1,0>|0,1>
|0,1>|1,0>
|0,1>|0,1>



# Hubbard Model Part II & III

In [157]:
from numpy import zeros as emptyM #empty matrix
from numpy.linalg import eigh

#The following program is designed for 1-Dimensional Hubbard Models with open bounds

sampleHilb = [[1,0,1,0],
              [0,1,0,1],
              [0,1,1,0],
              [1,0,0,1],] # 2 sites, half filled

# sampleHilb = [[1,1,0,1],
#               [1,1,1,0]] # 2 sites, 3 electrons

# sampleHilb = [[1,1,1,0,1,1],
#               [1,1,1,1,1,0],
#               [1,1,1,1,0,1]] # 3 sites, 5 electrons


#Declare parameters
t = 1
U = 1
hilbertSpace = sampleHilb #change from sampleHilb

# Create empty Hamiltonian matrix
ham = emptyM([len(hilbertSpace),len(hilbertSpace)])

# Onsite term------

#Counting Operator for individual bitstring
def countingOp(bitString, spin, site):
    # reduce bitstring to sites in question
    if(spin == "u"):
        siteFind = bitString[0:siteNum]
    else:
        siteFind = bitString[siteNum:]
    # analyze fermion presence at site
    if(siteFind[site] == 0):
        return(0)
    else:
        return(1)

#Compute coefficient for individual onsite term ket
def computeOnsiteTerm(bitString, numSites, U):
    coefficient = 0
    for n in range(numSites):
        if(countingOp(bitString,"u",n) == 1 and countingOp(bitString,"d",n) == 1):
            coefficient += 1
    return [coefficient*U,bitString]

#onsite terms for each bit string
#format: list index --> tuple [coefficient, bitString]
        #list index represents bitString index in hilbert space
u_terms = []
for bitString in hilbertSpace:
    u_terms.append(computeOnsiteTerm(bitString.copy(), siteNum, U))


# Hopping term------

#format: list index --> tuple [ [coefficient, modified up bitString] , [coefficient, modified down bitString] ]
t_terms = []
for ind in range(len(hilbertSpace)):
    for i in range(siteNum-1):
        coefficient1 = 0
        coefficient2 = 0
        
        newB_1 = hilbertSpace[ind].copy()
        newB_2 = hilbertSpace[ind].copy()
        j = i+1
        
        t_terms.append([[0],[0]])
        
        #Spin up j annihilate, spin up i create
        if(countingOp(newB_1,"u",j) == 1 and countingOp(newB_1,"u",i) == 0):
            newB_1[j] = 0
            newB_1[i] = 1
            coefficient1 += 1
            
            #fermionic phase statistic
            for f in range(i): 
                if(countingOp(newB_1,"u",i) == 1):
                    coefficient1 *= -1
                    
            t_terms[ind][0] = [coefficient1*t, newB_1]

        #Spin down j annihilate, spin down i create
        if(countingOp(newB_2,"d",j) == 1 and countingOp(newB_2,"d",i) == 0):
            newB_2[j+siteNum] = 0
            newB_2[i+siteNum] = 1
            coefficient2 += 1
            
            #fermionic phase statistic
            for f in range(i): 
                if(countingOp(newB_2,"d",i) == 1):
                    coefficient2 *= -1
                    
            t_terms[ind][1] = [coefficient2*t, newB_2]

# inner product function
def innerProd(b,k):
    for i in range(len(k)):
        if(k[i] != b[i]):
            return(0)
    return(1)


c = 0
# compute all inner products (u term)
for i in range(len(hilbertSpace)):
    for j in range(len(hilbertSpace)):
        #add in u term values
        if(u_terms[j][0] != 0):
            ham[i,j] += u_terms[i][0]*innerProd(hilbertSpace[j], u_terms[i][1]) #<j|i>
        #add in t term values
        for b in t_terms[i]:
            if(b != [0]):
                ham[i,j] += b[0]*innerProd(hilbertSpace[j], b[1])

# print(u_terms)
# print(t_terms)
# print(ham)

#diagonalize matrix
ground = min(eigh(ham)[0])

print("Hamiltonian matrix: \n"+str(ham))
print("Ground state energy = "+str(ground))

Hamiltonian matrix: 
[[1. 0. 0. 0.]
 [0. 1. 1. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]
Ground state energy = -1.0000000000000002


# Hubbard Model Part IV: Qiskit Attempt

In [2]:
from qiskit_nature.problems.second_quantization.lattice import LineLattice, FermiHubbardModel
import networkx as nx
import numpy as np

In [61]:
#U = onsite_parameter and t = edge_parameter
lattice = LineLattice(4,onsite_parameter=1.3,edge_parameter=0.9)

onsite = 2 #onsite interaction strength

hopping = np.array([[4.8, 0.2],[5.9, 2.1]],dtype=float)

hamiltonian = FermiHubbardModel(lattice, 2).second_q_ops(display_format="sparse")
print(hamiltonian)
print()

hamiltonian = FermiHubbardModel(lattice, 2).from_parameters(hopping,2.4).hopping_matrix()
print(hamiltonian)
print()

hamiltonian = FermiHubbardModel(lattice, 2).hopping_matrix()
print(hamiltonian)

Fermionic Operator
register length=8, number terms=24
  (0.9+0j) * ( +_0 -_2 )
+ (-0.9+0j) * ( -_0 +_2 )
+ (0.9+0j) * ( +_2 -_4 )
+ (-0.9+0j) * ( -_2 +_4 )
+ (0.9+0j) * ( +_4 -_6 )
+ (-0.9+0j) * ( -_4 +_6 )
+ (1.3+0j) * ( +_0 -_0 )
+ (1.3+0j) * ( +_2 ...

[[4.8+0.j 0.2+0.j]
 [0.2+0.j 2.1+0.j]]

[[1.3+0.j 0.9+0.j 0. +0.j 0. +0.j]
 [0.9+0.j 1.3+0.j 0.9+0.j 0. +0.j]
 [0. +0.j 0.9+0.j 1.3+0.j 0.9+0.j]
 [0. +0.j 0. +0.j 0.9+0.j 1.3+0.j]]
